# Folium Package Presentation

#### Mary Grace McClellan, GIS 5103

## Demo

### Basemap basic configuration -- folium.Map, folium.Popup

Folium can create an interactive basemap (otherwise known as a [Leaflet](https://leafletjs.com/) map). Latitude and longitude arguments can be passed to the Map function, or it can be left blank, as shown below:

In [181]:
import folium
m = folium.Map()
m

Adding latitude and longitude to the `location` argument will create a basemap of that location. Passing a number to the `zoom_start` argument will zoom in to the location. The basemap defaults to [OpenStreetMap](https://www.openstreetmap.org/#map=4/38.01/-95.84) tiles, but Folium has other tiles (such as [Stamen](http://maps.stamen.com)). In the example below, I'm trying out different tiles as well as plotting the location of an establishment in Amsterdam.  

Other default tiles to try:
 - Stamen Watercolor
 - Stamen Toner
 - Mapbox Bright

In [185]:
amst = folium.Map(location = [52.3680, 4.9036],
                 tiles = "Stamen Terrain",
                 zoom_start = 13) 

tooltip = 'When in Amsterdam, go to an Irish pub' #Text to display when hovering over marker
popup = folium.Popup("Molly Malone\'s Irish Pub", parse_html = True) #Text to display when marker is selected
folium.Marker(location=[52.376046, 4.901262], popup = popup, tooltip = tooltip).add_to(amst) #Marker configuration

amst

I think the idea of going to a different country to try a third country's cuisine is pretty funny. Let's get the locations of all the Irish pubs in Amsterdam using the Yelp API.

In [188]:
from yelpapi import YelpAPI
import pandas as pd
import numpy as np 

#The Yelp API key is the one that was used in Exercise 6, this should be unique to you

client_id = '8wOqTvvUmsZcLRz-CjckMg'
api_key = 'wxJ03UfeZV1R6CJXYmBIFy6vzVN_RNzeIJXZuAIAhvowI4enWmKxEuczr_HLeAI11YTUBxxxk8Iy3H7XZFiGZ0LGPhDqP1oxYliKLZj8gmgkEzpIyJnm0proiTLGW3Yx'
yelp_api = YelpAPI(api_key)

irishpubs = yelp_api.search_query(term='Irish Pub', location='Amsterdam', limit=50)

ptnames=[]
ptlats=[]
ptlongs=[]
ptkeys=[]

for i in range(len(irishpubs['businesses'])):
    ptnames.append(irishpubs['businesses'][i]['name'])
    ptlats.append(irishpubs['businesses'][i]['coordinates']['latitude'])
    ptlongs.append(irishpubs['businesses'][i]['coordinates']['longitude']) 
    ptkeys.append(irishpubs['businesses'][i]['id'])
    
pubs = {'Name': ptnames, 'Latitude': ptlats, 'Longitude':ptlongs, 'Key':ptkeys}

pints = pd.DataFrame(data=pubs)

pd.DataFrame.dropna(pints, inplace = True)
pints.reset_index(drop=True, inplace = True)


 ### More map customization -- folium.Icon, folium.LatLngPopup, and fit_bounds

Passing a number to the `zoom_start` argument isn't always the best way to frame the data. In the code below, we will use `fit_bounds` to create a bounding box. To get the bounding box coordinates, we can use `LatLngPopup()`, which shows the latitude/longitude of any place you click on the map. You can also see how to customize the markers using the [Font Awesome](https://fontawesome.com/icons) icon library. 

<i>(Try adding "fa-spin" after the name of the icon you choose, for instance `icon='beer fa-spin'`)</i>

In [179]:
#Set basemap
amster= folium.Map(location = [52.3680, 4.9036],
                  tiles = "CartoDB positron", #yet another default basemap
                  zoom_start = 13)

names = pints.iloc[:,0]
names = list(names)

for pt in range(0, len(pints.index)):
    folium.Marker(location = [pints.iloc[pt,1], pints.iloc[pt,2]], 
                  popup = (folium.Popup(names[pt], parse_html = True)), 
                  icon = folium.Icon(color='green', icon ='beer', prefix='fa')).add_to(amster)


amster.add_child(folium.LatLngPopup())
amster

In [180]:
amster.fit_bounds([[52.3663, 4.9090],[52.3782, 4.8865]]) #A list of two points, southeast and northwest
                                                            # corners of the bounding box
amster